In [2]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 38.3 MB/s eta 0:00:0000:0100:01


In [3]:
import torch
import json
import torch.onnx
import onnx
import onnxruntime as ort
import numpy as np

In [4]:
# Load configuration
with open("./pretrain/config.json", "r") as f:
    config = json.load(f)
    
config

{'resblock': '2',
 'num_gpus': 0,
 'batch_size': 16,
 'learning_rate': 0.0002,
 'adam_b1': 0.8,
 'adam_b2': 0.99,
 'lr_decay': 0.999,
 'seed': 1234,
 'upsample_rates': [8, 8, 4],
 'upsample_kernel_sizes': [16, 16, 8],
 'upsample_initial_channel': 256,
 'resblock_kernel_sizes': [3, 5, 7],
 'resblock_dilation_sizes': [[1, 2], [2, 6], [3, 12]],
 'resblock_initial_channel': 128,
 'segment_size': 8192,
 'num_mels': 80,
 'num_freq': 1025,
 'n_fft': 1024,
 'hop_size': 256,
 'win_size': 1024,
 'sampling_rate': 22050,
 'fmin': 0,
 'fmax': 8000,
 'fmax_loss': None,
 'num_workers': 4,
 'dist_config': {'dist_backend': 'nccl',
  'dist_url': 'tcp://localhost:54321',
  'world_size': 1}}

In [ ]:
import json
with open("./pretrain/config.json") as f:
    config = json.load(f)

print(config)

# Load config first (usually stored in a .json file)
dict(config)

{'resblock': '2', 'num_gpus': 0, 'batch_size': 16, 'learning_rate': 0.0002, 'adam_b1': 0.8, 'adam_b2': 0.99, 'lr_decay': 0.999, 'seed': 1234, 'upsample_rates': [8, 8, 4], 'upsample_kernel_sizes': [16, 16, 8], 'upsample_initial_channel': 256, 'resblock_kernel_sizes': [3, 5, 7], 'resblock_dilation_sizes': [[1, 2], [2, 6], [3, 12]], 'resblock_initial_channel': 128, 'segment_size': 8192, 'num_mels': 80, 'num_freq': 1025, 'n_fft': 1024, 'hop_size': 256, 'win_size': 1024, 'sampling_rate': 22050, 'fmin': 0, 'fmax': 8000, 'fmax_loss': None, 'num_workers': 4, 'dist_config': {'dist_backend': 'nccl', 'dist_url': 'tcp://localhost:54321', 'world_size': 1}}


{'resblock': '2',
 'num_gpus': 0,
 'batch_size': 16,
 'learning_rate': 0.0002,
 'adam_b1': 0.8,
 'adam_b2': 0.99,
 'lr_decay': 0.999,
 'seed': 1234,
 'upsample_rates': [8, 8, 4],
 'upsample_kernel_sizes': [16, 16, 8],
 'upsample_initial_channel': 256,
 'resblock_kernel_sizes': [3, 5, 7],
 'resblock_dilation_sizes': [[1, 2], [2, 6], [3, 12]],
 'resblock_initial_channel': 128,
 'segment_size': 8192,
 'num_mels': 80,
 'num_freq': 1025,
 'n_fft': 1024,
 'hop_size': 256,
 'win_size': 1024,
 'sampling_rate': 22050,
 'fmin': 0,
 'fmax': 8000,
 'fmax_loss': None,
 'num_workers': 4,
 'dist_config': {'dist_backend': 'nccl',
  'dist_url': 'tcp://localhost:54321',
  'world_size': 1}}

In [6]:
from models import Generator

In [10]:
import importlib
import models
importlib.reload(models)
from models import Generator

In [12]:
generator = Generator(config)

sfsdf {'resblock': '2', 'num_gpus': 0, 'batch_size': 16, 'learning_rate': 0.0002, 'adam_b1': 0.8, 'adam_b2': 0.99, 'lr_decay': 0.999, 'seed': 1234, 'upsample_rates': [8, 8, 4], 'upsample_kernel_sizes': [16, 16, 8], 'upsample_initial_channel': 256, 'resblock_kernel_sizes': [3, 5, 7], 'resblock_dilation_sizes': [[1, 2], [2, 6], [3, 12]], 'resblock_initial_channel': 128, 'segment_size': 8192, 'num_mels': 80, 'num_freq': 1025, 'n_fft': 1024, 'hop_size': 256, 'win_size': 1024, 'sampling_rate': 22050, 'fmin': 0, 'fmax': 8000, 'fmax_loss': None, 'num_workers': 4, 'dist_config': {'dist_backend': 'nccl', 'dist_url': 'tcp://localhost:54321', 'world_size': 1}}


In [17]:
checkpoint = torch.load("./pretrain/generator_v3", map_location='cpu')
generator.load_state_dict(checkpoint['generator'])
generator.eval()

/var/folders/jp/rdbm5vdn37d4r9b4jfc_k23m0000gn/T/ipykernel_88734/1877114018.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("./pretrain/generator

Generator(
  (conv_pre): Conv1d(80, 256, kernel_size=(7,), stride=(1,), padding=(3,))
  (ups): ModuleList(
    (0): ConvTranspose1d(256, 128, kernel_size=(16,), stride=(8,), padding=(4,))
    (1): ConvTranspose1d(128, 64, kernel_size=(16,), stride=(8,), padding=(4,))
    (2): ConvTranspose1d(64, 32, kernel_size=(8,), stride=(4,), padding=(2,))
  )
  (resblocks): ModuleList(
    (0): ResBlock2(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
      )
    )
    (1): ResBlock2(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
        (1): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(12,), dilation=(6,))
      )
    )
    (2): ResBlock2(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
        (1): Conv1

In [18]:
import onnxruntime
print(onnxruntime.get_device())

CPU
